![](supplychain.png)
# Ledger - Trusted Supply Chain Demo - SQL Server 2022



## Append-Only Ledger Table Shipments

In [ ]:
CREATE TABLE [Shipments]
(
    [ShipmentID] INT NOT NULL PRIMARY KEY CLUSTERED IDENTITY(1,1),
	[ShipmentNumber] INT NULL,
	[Responsibility] NVARCHAR(50),
	[Temperature] INT,
    [OutOfCompliance] BIT,
	[CountryName] VARCHAR(50),
    [LastModified] [Datetime2] NOT NULL DEFAULT (SYSDATETIMEOFFSET())
)
WITH 
(
  LEDGER = ON (APPEND_ONLY = ON)
);
GO

![](compliance.png)

## Let's try to modify the data that is out of compliance.

The shipment responsible knows the DBA of Contoso IC and asks to manipulate the data since his company made a mistake and wants to avoid huge claims.  
<span style="font-family: Calibri, sans-serif; font-size: 11pt;">The DBA tries to manipulate the faulty record but noticed that it’s an append-only ledger table and that data cannot be modified.&nbsp;</span>

In [1]:
SELECT * from Shipments
WHERE OutOfCompliance=1

(1 row affected)

Total execution time: 00:00:00.095

ShipmentID,ShipmentNumber,Responsibility,Temperature,OutOfCompliance,CountryName,LastModified
310,94,Contoso East Shipping,10,1,Belgium,2021-07-03 12:37:07.0000000


In [22]:
UPDATE Shipments
SET Temperature=-1, OutOfCompliance=0
WHERE ShipmentID=312

: Msg 37359, Level 16, State 1, Line 1
Updates are not allowed for the append only Ledger table 'Shipments'.

Total execution time: 00:00:00.063

## Updateable Ledger Table ShipMethod

In [4]:
CREATE TABLE [ShipMethod](
	[ShipMethodID] INT,
	[Name] NVARCHAR(256) NOT NULL,
	[ShipBase] MONEY NOT NULL,
	[ShipRate] MONEY NOT NULL,
	[rowguid] UNIQUEIDENTIFIER ROWGUIDCOL  NOT NULL
	)
WITH 
(
  SYSTEM_VERSIONING = ON,
  LEDGER = ON
); 
GO

Commands completed successfully.

Total execution time: 00:00:00.057

## View the history of ShipMethod Table

In [5]:
	SELECT 
	t.[commit_time] AS [CommitTime] 
	, t.[principal_name] AS [UserName]
	, l.[ShipMethodID]
	, l.[Name]
	, l.[ShipBase]
	, l.[ShipRate]
	, l.[rowguid]
	, l.[ledger_operation_type_desc] AS Operation
	FROM [dbo].[ShipMethod_Ledger] l
	JOIN sys.database_ledger_transactions t
	ON t.transaction_id = l.ledger_transaction_id
	WHERE t.[commit_time] > DATEADD(MINUTE, -10, SYSDATETIMEOFFSET())
	ORDER BY t.commit_time DESC;

(7 rows affected)

Total execution time: 00:00:00.074

CommitTime,UserName,ShipMethodID,Name,ShipBase,ShipRate,rowguid,Operation
2022-04-15 11:45:30.8900000,PowerApp,3,OVERSEAS - DELUXE,29.95,1.99,22f4e461-28cf-4ace-a980-f686cf112ec8,INSERT
2022-04-15 11:45:30.8900000,PowerApp,3,OVERSEAS - DELUXE,29.95,2.99,22f4e461-28cf-4ace-a980-f686cf112ec8,DELETE
2022-04-15 11:40:29.6300000,PowerApp,1,XRQ - TRUCK GROUND,3.95,0.99,6be756d9-d7be-4463-8f2c-ae60c710d606,INSERT
2022-04-15 11:40:29.6300000,PowerApp,2,ZY - EXPRESS,9.95,1.99,3455079b-f773-4dc6-8f1e-2a58649c4ab8,INSERT
2022-04-15 11:40:29.6300000,PowerApp,4,OVERNIGHT J-FAST,21.95,1.29,107e8356-e7a8-463d-b60c-079fff467f3f,INSERT
2022-04-15 11:40:29.6300000,PowerApp,5,CARGO TRANSPORT,8.99,1.49,b166019a-b134-4e76-b957-2b0490c610ed,INSERT
2022-04-15 11:40:29.6300000,PowerApp,3,OVERSEAS - DELUXE,29.95,2.99,22f4e461-28cf-4ace-a980-f686cf112ec8,INSERT


## Different parties can verify the database to be sure the data can be trusted.

In [25]:
EXECUTE [dbo].[sp_verify_Manual_Digest] 'F:\Digests\Digests.txt'

Ledger verification successfully verified up to block 0.

Total execution time: 00:00:00.127

last_verified_block_id
0


## Let's tamper the data

DBA thinks he/she’s smart and tampers with the data directly into the page by using a stored procedure.

In [26]:
SELECT TOP 10 sys.fn_PhysLocFormatter(%%physloc%%) PageId, *
FROM Shipments
WHERE ShipmentID=312 --Copy the ID from the previous result set

(1 row affected)

Total execution time: 00:00:00.057

PageId,ShipmentID,ShipmentNumber,Responsibility,Temperature,OutOfCompliance,CountryName,LastModified
(1:285:35),312,95,Contoso East Shipping,10,1,Belgium,2021-07-03 12:36:57.0000000


In [ ]:
EXECUTE sp_TamperTemperature

Show that the high temperature is gone

In [28]:
SELECT * from Shipments
WHERE OutOfCompliance=1

(0 rows affected)

Total execution time: 00:00:00.052

ShipmentID,ShipmentNumber,Responsibility,Temperature,OutOfCompliance,CountryName,LastModified


## Verify the database again

In [29]:
EXECUTE [dbo].[sp_verify_Manual_Digest] 'F:\Digests\Digests.txt'

: Msg 37371, Level 16, State 1, Procedure sp_verify_database_ledger, Line 1
The computed hash from 'Shipments' and the associated history table does not match the hash persisted in sys.database_ledger_transactions for transaction 1413.

: Msg 37392, Level 16, State 1, Procedure sp_verify_database_ledger, Line 1
Ledger verification failed.

Total execution time: 00:00:00.136

## Review the history of the transaction

In [31]:
SELECT
	t.[commit_time] AS [CommitTime] 
	, t.[principal_name] AS [UserName]
	, l.[ShipmentID]
	, l.[ShipmentNumber]
	, l.[Responsibility]
	, l.[Temperature]
	, l.[OutOfCompliance]
    , l.[CountryName]
	, l.[ledger_operation_type_desc] AS Operation
	FROM [dbo].[Shipments_Ledger] l
	JOIN [sys].[database_ledger_transactions] t
	ON t.[transaction_id] = l.[ledger_transaction_id]
	WHERE t.transaction_id=1413;

(1 row affected)

Total execution time: 00:00:00.062

CommitTime,UserName,ShipmentID,ShipmentNumber,Responsibility,Temperature,OutOfCompliance,CountryName,Operation
2022-04-14 08:58:43.2966667,PowerApp,312,95,Contoso East Shipping,-1,0,Belgium,INSERT
